In [95]:
import pandas as pd
from string import Template
from unidecode import unidecode

# Load data

In [116]:
first_year = 20
last_year = 23
# File name and path
file_name = Template("Source_data/negociacao-20$year-01-01-a-20$year-$last_date.xlsx")
# Load files
files_dict = {}
for year in range(first_year,last_year+1):
    last_date = "12-31"
    if year == last_year:
        last_date = "03-16"
        
    files_dict[f"20{year}"] = pd.read_excel(file_name.substitute(year = year, last_date = last_date), sheet_name="Negociação", engine="openpyxl")

# Columns to be converted to datetime
dates_cols = ["Data do Negócio", "Prazo/Vencimento"]
# Convert columns to datetime
for key in range(first_year,last_year+1):
    temp = files_dict[f"20{key}"]
    temp[dates_cols] = temp[dates_cols].apply(pd.to_datetime, format="%d/%m/%Y", errors="coerce")
    temp["Corretagem"] = 0
    
files_dict["2020"].head(3)

c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's de

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor,Corretagem
0,2020-11-30,Compra,Mercado à Vista,2020-11-30,RICO INVESTIMENTOS - GRUPO XP,HGRU11,2,128.16,256.32,0
1,2020-11-30,Compra,Mercado à Vista,2020-11-30,RICO INVESTIMENTOS - GRUPO XP,KNRI11,2,159.80,319.60,0
2,2020-11-30,Compra,Mercado Fracionário,NaT,RICO INVESTIMENTOS - GRUPO XP,SAPR4F,9,5.10,45.90,0


# Add "Corretagem" columns to 2020 data

In [117]:
# Add 'corretagem' cost to some stocks in 2020
corretagem_string = """MGLU3 8,30 2020-03-13
BRKM5 8,30 2020-03-20
TRPL4 8,30 2020-03-20
ALZR11 1,40 2020-08-13
FLRY3 1,40 2020-08-13
HGLG11 1,40 2020-08-13
ITSA4 1,40 2020-08-13
KLBN4 1,40 2020-08-13
WEGE3 1,40 2020-08-13
BRKM5 2,10 2020-09-03
MGLU3 2,10 2020-09-03
WEGE3 2,10 2020-09-03"""
corretagem_list = [ elem.replace(",", ".").split(" ") for elem in corretagem_string.split("\n")]
df_corretagem = pd.DataFrame(corretagem_list, columns=["Código de Negociação", "Corretagem", "Data do Negócio"])
df_corretagem["Corretagem"] = df_corretagem["Corretagem"].astype(float)
df_corretagem["Data do Negócio"] = pd.to_datetime(df_corretagem["Data do Negócio"], format="%Y-%m-%d", errors="coerce")
df_corretagem.head(3)


,Código de Negociação,Corretagem,Data do Negócio
0,MGLU3,8.3,2020-03-13
1,BRKM5,8.3,2020-03-20
2,TRPL4,8.3,2020-03-20


In [118]:
for idx, (cod, val, dt) in df_corretagem.iterrows():
    mask = (files_dict["2020"]["Código de Negociação"].str.contains(cod)) & (files_dict["2020"]["Data do Negócio"] == dt)
    files_dict["2020"].loc[mask, "Corretagem"] = val

# Prepare data to save in csv

In [127]:
final_df = pd.DataFrame()
# Fix columns names
for key in range(first_year,last_year+1):
    files_dict[f"20{key}"].columns = [unidecode(ii.replace(" ", "_")).lower() for ii in files_dict[f"20{key}"].columns]
    # Save files
    final_df = pd.concat([final_df, files_dict[f"20{key}"]])
    
    
final_df.to_csv(f"Ready_to_load/Stocks/negociacao.csv", sep=";", encoding="UTF8", index=False)


# Load Proventos

In [154]:
first_year = 20
last_year = 23
# File name and path
file_name = Template("Source_data/proventos-recebidos-20$year-01-01-a-20$year-$last_date.xlsx")
# Load files
files_dict = {}
for year in range(first_year,last_year+1):
    last_date = "12-31"
    if year == last_year:
        last_date = "03-16"
        
    files_dict[f"20{year}"] = pd.read_excel(file_name.substitute(year = year, last_date = last_date), sheet_name="Proventos Recebidos", engine="openpyxl", skipfooter=3)

# Columns to be converted to float
for key in range(first_year,last_year+1):
    temp = files_dict[f"20{key}"]
    temp["Quantidade"] = temp["Quantidade"].str.replace(",", ".").astype(float)
    temp["Pagamento"] = temp["Pagamento"].apply(pd.to_datetime, format="%d/%m/%Y", errors="coerce")
    
files_dict["2020"].head(3)

c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ivana\miniconda3\envs\home_data_architecture\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's de

,Produto,Pagamento,Tipo de Evento,Instituição,Quantidade,Preço unitário,Valor líquido
0,TAEE4 - TRANSMISSORA ALIANCA DE ENERGIA ELETRI...,2020-12-28,Juros Sobre Capital Próprio,RICO INVESTIMENTOS - GRUPO XP,6.0,0.05,0.27
1,ALZR11 - ALIANZA TRUST RENDA IMOBILIARIA - FUN...,2020-12-23,Rendimento,RICO INVESTIMENTOS - GRUPO XP,1.0,0.38,0.37
2,XPML11 - XP MALLS FDO INV IMOB FII,2020-12-23,Rendimento,RICO INVESTIMENTOS - GRUPO XP,3.0,0.45,1.35


In [158]:
final_df = pd.DataFrame()
# Fix columns names
for key in range(first_year,last_year+1):
    files_dict[f"20{key}"].columns = [unidecode(ii.replace(" ", "_")).lower() for ii in files_dict[f"20{key}"].columns]
    # Save files
    final_df = pd.concat([final_df, files_dict[f"20{key}"]])
    
final_df.to_csv(f"Ready_to_load/Stocks/proventos.csv", encoding="UTF8", index=False)

In [157]:
final_df

,produto,pagamento,tipo_de_evento,instituicao,quantidade,preco_unitario,valor_liquido
0,TAEE4 - TRANSMISSORA ALIANCA DE ENERGIA ELETRI...,2020-12-28,Juros Sobre Capital Próprio,RICO INVESTIMENTOS - GRUPO XP,6.0,0.05,0.27
1,ALZR11 - ALIANZA TRUST RENDA IMOBILIARIA - FUN...,2020-12-23,Rendimento,RICO INVESTIMENTOS - GRUPO XP,1.0,0.38,0.37
2,XPML11 - XP MALLS FDO INV IMOB FII,2020-12-23,Rendimento,RICO INVESTIMENTOS - GRUPO XP,3.0,0.45,1.35
3,HGLG11 - CSHG LOGISTICA FUNDO DE INVESTIMENTO ...,2020-12-14,Rendimento,RICO INVESTIMENTOS - GRUPO XP,1.0,0.78,0.78
4,HGRU11 - CSHG RENDA URBANA - FUNDO DE INVESTIM...,2020-12-14,Rendimento,RICO INVESTIMENTOS - GRUPO XP,2.0,0.68,1.36
...,...,...,...,...,...,...,...
27,HGRU11 - CSHG RENDA URBANA - FUNDO DE INVESTIM...,2023-01-13,Rendimento,XP INVESTIMENTOS CCTVM S/A,3.0,2,6.00
28,KNRI11 - KINEA RENDA IMOBILIÁRIA FDO INV IMOB ...,2023-01-13,Rendimento,XP INVESTIMENTOS CCTVM S/A,3.0,1,3.00
29,MXRF11 - MAXI RENDA FUNDO DE INVESTIMENTO IMOB...,2023-01-13,Rendimento,XP INVESTIMENTOS CCTVM S/A,42.0,0.1,4.20
30,XPCI11 - XP CREDITO IMOBILIÁRIO - FDO INV IMOB,2023-01-13,Rendimento,XP INVESTIMENTOS CCTVM S/A,3.0,0.72,2.16
